In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import math

In [ ]:
data = pd.read_csv('train.csv')

In [ ]:
data = np.array(data)
m, n = data.shape
np.random.shuffle(data)

data_dev = data[0:1000].T
Y_dev = data_dev[0]
X_dev = data_dev[1:n]

data_train = data[1000:m].T
Y_train = data_train[0]
X_train = data_train[1:n]


In [4]:
w_1 = np.random.rand(10, 784)
w_2 = np.random.rand(10, 10)
b_1 = np.random.rand(10,1)
dw_1 = np.ones((10, 784))
dw_2 = np.ones((10, 10))
db_1 = np.ones(10)

def relu(x):
    return (x>0) * x
    
def drelu(x):
    return 1 * (x > 0)

def sigmoid(x):
    return 1/(1 + (math.e)**(-x)), 1, 'a'

def feedforward(w_1, w_2, b_1, a_0):
    a_1 = np.matmul(w_1, a_0)
    a_1 += b_1.reshape((10,))
    z_1 = a_1
    for i in range(10):
        a_1[i] = relu(a_1[i])
    a_2 = np.matmul(w_2, a_1)
    return z_1,a_1,a_2

def cost(w_1, w_2, b_1, i):
    sum = 0
    temp = np.zeros(10)
    temp[Y_train[i]] = 1
    sum += np.sum((temp - feedforward(w_1, w_2, b_1, X_train[:, i])[2]) ** 2)
    return sum

NameError: name 'np' is not defined

In [2]:
def epoch(alpha,w_1, w_2, b_1,dw_1,dw_2,db_1):
    for i in range(4100):
        z_1,a_1,a_2 = feedforward(w_1,w_2,b_1,X_train[:,i])
        for x in range(10):
            temp = 0
            for y in range(10):
                temp += w_2[y][x]*drelu(z_1[x])*(a_2[y]-(y==Y_train[i]))
            db_1[x] = temp
        for x in range(10):
            for y in range(10):
                dw_2[y][x] = a_1[x] * (a_2[y]-(y==Y_train[i]))
        for x in range(784):
            for y in range(10):
                temp = 0
                for k in range(10):
                    temp += X_train[x][i]*drelu(a_1[y])*dw_2[k][y]
                dw_1[y][x] = temp
        w_1 -= alpha * dw_1
        w_2 -= alpha * dw_2
        b_1 -= alpha * db_1.reshape(10,1)
    return w_1,w_2,b_1
            

In [3]:
for i in range(1):
    w_1, w_2, b_1 = epoch(0.001, w_1, w_2, b_1, dw_1, dw_2, db_1)
    
cost(w_1,w_2,b_1,10)

NameError: name 'w_1' is not defined